Created on Thu Apr 18 05:43:21 2024

@author: Santiago D'hers

Use:

- This script will create autolabels analyzing position files

Requirements:

- The position.csv files processed by 1-Manage_H5.py
- The desired model trained with 3a-Create_Models.py

In [4]:
import os
from glob import glob
import pandas as pd
import numpy as np

import tensorflow as tf

import joblib
from keras.models import load_model

import datetime

In [5]:
# Set the variables before starting
desktop = 'C:/Users/dhers/Desktop'
STORM_folder = os.path.join(desktop, 'STORM/models')

# State your path:
path = r'C:\Users\dhers\OneDrive - UBA\workshop'
experiment = r'Ultron'

frames = 7

all_position = glob(os.path.join(path, experiment,"T*/position/*position.csv"))

today = datetime.datetime.now()
# use_model_date = today.date()
use_model_date = '2024-10-22'

objects = ['obj_1', 'obj_2']
bodyparts = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body']
rescale = True
reshape = True # True for wide models

In [6]:
# Load the saved model from file

# loaded_model = load_model(os.path.join(STORM_folder, f'simple/model_simple_{use_model_date}.keras'))
loaded_model = load_model(os.path.join(STORM_folder, f'wide/model_wide_{use_model_date}.keras'))
# loaded_model = joblib.load(os.path.join(STORM_folder, f'RF/model_RF_{use_model_date}.pkl'))

In [7]:
def recenter(df, point, bodyparts):
    # Create a copy of the original dataframe
    df_copy = df.copy()
    bodypart_columns = []
    
    for bodypart in bodyparts:
        # Subtract point_x from columns ending in _x
        x_cols = [col for col in df_copy.columns if col.endswith(f'{bodypart}_x')]
        df_copy[x_cols] = df_copy[x_cols].apply(lambda col: col - df_copy[f'{point}_x'])
        
        # Subtract point_y from columns ending in _y
        y_cols = [col for col in df_copy.columns if col.endswith(f'{bodypart}_y')]
        df_copy[y_cols] = df_copy[y_cols].apply(lambda col: col - df_copy[f'{point}_y'])
        
        # Collect bodypart columns
        bodypart_columns.extend(x_cols)
        bodypart_columns.extend(y_cols)
        
    return df_copy[bodypart_columns]

In [8]:
def reshape(df):
    reshaped_df = []
    
    # Iterate over each row index in the DataFrame
    for i in range(len(df)):
        # Determine which indices to include for reshaping
        indices_to_include = [
            max(0, i - 6),  # i - 6, ensure it's not negative
            max(0, i - 3),  # i - 3, ensure it's not negative
            max(0, i - 1),  # i - 1, ensure it's not negative
            i,              # current index
            min(len(df) - 1, i + 1),  # i + 1, ensure it's not out of bounds
            min(len(df) - 1, i + 3),  # i + 3, ensure it's not out of bounds
            min(len(df) - 1, i + 6)   # i + 6, ensure it's not out of bounds
        ]
        
        # Append the rows using the calculated indices
        reshaped_df.append(df.iloc[indices_to_include])
    
    return reshaped_df

In [9]:
def use_model(position, model, objects = ['obj_1', 'obj_2'], bodyparts = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body'], recentering = True, reshaping = False):
    
    if recentering:
        dfs = []
        for obj in objects:
            recentered = recenter(position, obj, bodyparts)
            dfs.append(recentered)
        position = pd.concat(dfs,ignore_index=True)
    
    if reshaping:
        position = np.array(reshape(position))
    
    pred = model.predict(position) # Use the model to predict the labels
    pred = pred.flatten()
    pred = pd.DataFrame(pred, columns=['predictions'])

    n_objects = len(objects)

    # Calculate the length of each fragment
    fragment_length = len(pred) // n_objects

    # Create a list to hold each fragment
    fragments = [pred.iloc[i*fragment_length:(i+1)*fragment_length].reset_index(drop=True) for i in range(n_objects)]

    # Concatenate fragments along columns
    labels = pd.concat(fragments, axis=1)

    # Rename columns
    labels.columns = [f'{obj}' for obj in objects]

    labels = round(labels, 2)
    
    return labels

In [10]:
def create_autolabels(files, model, objects = ['obj_1', 'obj_2'], bodyparts = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body'], rescaling = True, reshaping = False):
    
    for file in files:
        
        # Determine the output file path
        input_dir, input_filename = os.path.split(file)
        parent_dir = os.path.dirname(input_dir)
        
        # Read the file
        position = pd.read_csv(file)
    
        # lets analyze it!
        autolabels = use_model(position, model, objects, bodyparts, rescaling, reshaping)
        
        # Set column names and add a new column "Frame" with row numbers
        autolabels.insert(0, "Frame", autolabels.index + 1)
    
        # Create a filename for the output CSV file
        output_filename = input_filename.replace('_position.csv', '_autolabels.csv')
        output_folder = os.path.join(parent_dir + '/autolabels')
        os.makedirs(output_folder, exist_ok = True)
        output_path = os.path.join(output_folder, output_filename)
        
        # Save autolabels to a CSV file
        autolabels.to_csv(output_path, index=False)
        print(f"Saved autolabels to {output_filename}")

In [11]:
create_autolabels(all_position, loaded_model, objects, bodyparts, rescaling = rescale, reshaping = reshape) # Lets analyze!

469/469 [==============================] - 4s 6ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C1_A_L_autolabels.csv
457/457 [==============================] - 3s 6ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C1_A_R_autolabels.csv
465/465 [==============================] - 2s 4ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C1_B_L_autolabels.csv
464/464 [==============================] - 2s 5ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C1_B_R_autolabels.csv
469/469 [==============================] - 2s 5ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C2_A_L_autolabels.csv
465/465 [==============================] - 3s 6ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C2_A_R_autolabels.csv
460/460 [==============================] - 3s 6ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C2_B_L_autolabels.csv
460/460 [==============================] - 3s 6ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C2_B_R_autolabels.csv
469/469 [==============================] - 3s 6ms/step
Saved autolabels to 2023-